# HOW TO obtain archival ESO data given RA and Dec

In [1]:
%matplotlib notebook

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from ESOAsg.ancillary import astro
from ESOAsg.core import archive_observations
from astropy.coordinates import SkyCoord

## Get RA and Dec of one target (or more)

To begin, you need to choose the targets you would like to explore and get their coordinates. These needs to be an [astropy.coordinates](https://docs.astropy.org/en/stable/coordinates/) `SkyCoord` object. A `SkyCoord` object can easily be created by using:
```python
from astropy import units as u
from astropy.coordinates import SkyCoord
coordinates = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')
```
For this example, we consider a couple of standard stars observed as part of the [`MUSE library of stellar spectra`](https://www.eso.org/sci/publications/announcements/sciann17288.html) project ([Ivanov et al. 2019](https://ui.adsabs.harvard.edu/abs/2019A%26A...629A.100I/abstract)): 
* [HD 057060](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=HD+057060&submit=SIMBAD+search)
* [HD 172230](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=HD+172230&submit=SIMBAD+search).

The coordinates of these two stars are can be retrieved from their name using `Sesame`:

In [14]:
HD057060_position = SkyCoord.from_name('HD 057060')
HD172230_position = SkyCoord.from_name('HD 172230')
print('HD 057060: {}'.format(HD057060_position.to_string('hmsdms')))
print('HD 172230: {}'.format(HD172230_position.to_string('hmsdms')))

HD 057060: 07h18m40.3797s -24d33m31.3118s
HD 172230: 18h38m54.9544s +06d16m14.8348s


## Query the archive

There are several ways to query the ESO archive for the so-called Archive Data Products (in short, ADPs).

```SQL
SELECT
    target_name, dp_id, s_ra, s_dec, t_exptime, em_min, em_max, 
    dataproduct_type, instrument_name, obstech, abmaglim,
    proposal_id, obs_collection
FROM
    ivoa.ObsCore
WHERE
    CONTAINS(s_region,CIRCLE('ICRS',109.66824871,-24.55869773,5.0/3600.))=1
AND
    (instrument_name='MUSE' OR instrument_name='XSHOOTER')
```

In [61]:
position_list = [HD057060_position, HD172230_position]
result_from_queries = archive_observations.query_from_radec(position_list, radius=5., instruments=['MUSE', 'XSHOOTER'],
                                                           verbose=False)

[WORKING] :: Running query 1 to the ESO archive (out of 2 total)
[INFO]    :: A total of 4 entries has been retrieved
[WORKING] :: Running query 2 to the ESO archive (out of 2 total)
[INFO]    :: A total of 7 entries has been retrieved


The results are stored in a list of astropy tables (one table per position). For instance, for HD 057060:

In [55]:
result_from_queries[0].show_in_notebook(show_row_index=False)

target_name,dp_id,s_ra,s_dec,t_exptime,em_min,em_max,dataproduct_type,instrument_name,obstech,abmaglim,proposal_id,obs_collection
,,deg,deg,s,m,m,,,,mag,,
2MASS-J07184037-2433312,ADP.2017-06-02T19:41:04.876,109.668427,-24.55842,0.4,5.336599999999999e-07,1.02e-06,spectrum,XSHOOTER,"ECHELLE,SLIT,OFFSET",--,084.B-0869(B),XSHOOTER
2MASS-J07184037-2433312,ADP.2017-06-02T19:41:04.929,109.668427,-24.55842,4.0,9.9402e-07,2.47896e-06,spectrum,XSHOOTER,"ECHELLE,SLIT,OFFSET",--,084.B-0869(B),XSHOOTER
2MASS-J07184037-2433312,ADP.2017-06-02T19:41:04.937,109.668427,-24.55842,0.4,2.9892000000000003e-07,5.5598e-07,spectrum,XSHOOTER,"ECHELLE,SLIT,OFFSET",--,084.B-0869(B),XSHOOTER
HD 057060,ADP.2020-02-26T15:36:25.254,109.668254,-24.558703,0.9,4.74969e-07,9.34969e-07,spectrum,MUSE,IFU,--,099.D-0623(A),MUSE-STD


'ESO-specific field not present in the standard ObsCore. The original ESO identifier of the main science file. ObsCore uses obs_publisher_did, which returns the full IVO Identifier. This field can be used to join with other ESO (non-VO) tables, like provenance.'

## Query the ESO Archive to collect data withing the contours

Now it is time to run the proper query to the Archive to obtain all data located within the contour regions. To do this you have two options:
* Archive Science Portal: http://archive.eso.org/scienceportal/home
* Programmatic access: http://archive.eso.org/programmatic/

There is, however, one last step before doing this. You need to convert the contours into polygons. This is done with:
```python
download_archive.contours_to_polygons(contours, max_vertices=30)
```
where `max_vertices` is set to avoid crashes due to polygons with too many vertices in the queries.

In [6]:
polygons = download_archive.contours_to_polygons(contours, max_vertices=30)

The resulting `polygons` is a list with N elements (with N matching the number of contours). Each elements contains a string defining the location in the sky of the polygon with RA, Dec, seprated by commas and with the first RA, Dec pair that matches the last one (to close the polygon). For instance:

In [7]:
polygons[1]

'167.8711, -2.5374, 166.9922, -2.0894, 166.1133, -1.3430, 165.9375, -0.5968, 165.7617, 0.1492, 165.5859, 0.8953, 165.4102, 1.6415, 165.2344, 2.3880, 165.7617, 3.1349, 166.2891, 3.8824, 166.4648, 4.6305, 166.6406, 5.3794, 167.5195, 5.8292, 168.3984, 5.6792, 169.2773, 4.9300, 169.4531, 4.1815, 169.6289, 3.4338, 169.8047, 2.6867, 169.6289, 1.9401, 169.4531, 1.1938, 169.2773, 0.4476, 169.1016, -0.2984, 168.9258, -1.0445, 168.7500, -1.7908, 167.8711, -2.5374'

### Archive Science Portal Query:
After running:
```python
download_archive.query_ASP_from_polygons(polygons=polygons, open_link=True)
```
you should see different ASP pages opening in your broswer (one per polygon) pointing to the different regions of the sky you are quering.

In [8]:
download_archive.query_ASP_from_polygons(polygons=polygons, open_link=True)

### Programmatic access:
```python
download_archive.query_TAP_from_polygons(polygons=polygons)
```
returns the result of the query:
```SQL
SELECT
   target_name, dp_id, s_ra, s_dec, t_exptime, em_min, em_max, 
   dataproduct_type, instrument_name, abmaglim, proposal_id
FROM
   ivoa.ObsCore
WHERE
   intersects(s_region, POLYGON('', `polygon`)) = 1
```
for convenince `maxrec` is set to `2` so only the first two results are returned, but you may want to increase this value.

In [9]:
results_from_TAP = download_archive.query_TAP_from_polygons(polygons=polygons, 
                                                            merge=False, maxrec=2, 
                                                            verbose=False)

[INFO]    :: Querying the ESO TAP service at:
[INFO]    :: http://archive.eso.org/tap_obs
[INFO]    :: A total of 2 entries has been retrieved for polygon N.0
[INFO]    :: For the following instrument:
[INFO]    ::  - FEROS
[INFO]    ::  - HARPS
[INFO]    :: A total of 2 entries has been retrieved for polygon N.1
[INFO]    :: For the following instrument:
[INFO]    ::  - UVES
[INFO]    :: A total of 2 entries has been retrieved for polygon N.2
[INFO]    :: For the following instrument:
[INFO]    ::  - XSHOOTER


now you can download the data by exploring the `dp_id` of the results: 

In [10]:
download_archive.download(results_from_TAP[0]['dp_id'])

[INFO]    :: Your disk has: Total: 397.03 GB, Used: 397.03 GB, Free: 397.03 GB
[WARNING] :: The content of dp_id is not in a string format.
[WARNING] :: The code is trying to fix this.
[WARNING] :: Converted to <class 'str'>.
[WORKING] :: Downloading file ADP.2016-09-19T01:03:10.998.fits. This may take some time.
[INFO]    :: File ADP.2016-09-19T01:03:10.998.fits downloaded.
[WARNING] :: The content of dp_id is not in a string format.
[WARNING] :: The code is trying to fix this.
[WARNING] :: Converted to <class 'str'>.
[WORKING] :: Downloading file ADP.2016-09-19T11:46:29.074.fits. This may take some time.
[INFO]    :: File ADP.2016-09-19T11:46:29.074.fits downloaded.


## All in one
The same resuls are achievable with the (un-official) ESOAsg script:
```python
get_data_from_gw_event.py
```